# Colorspace - Patterns - Volumetric Graphs

Read in the ram history from our dump file.

In [1]:
log_file = "colorspace_presets_volumetric.txt"
input_file = open(log_file,'r')

flatten = lambda l: [e for sublist in l for e in sublist]

ram_history = []
lines = input_file.readlines()
for i in range(1, len(lines), 131):
    # Screen RAM (7000-8000)
    raw_ram = [l[6:126].split() for l in lines[i:i+104]]
    pixel_ram = [int(v,16) for l in raw_ram for v in l][:0x1000]
    # Display list (8000-9000)
    raw_ram = [l[6:126].split() for l in lines[i+103:i+127]]
    display_list = [int(v,16) for l in raw_ram for v in l]
    # Color Registers
    color_registers = [int(i,16) for i in lines[i+129].split()[1:10]]
    ram_history += [(pixel_ram, color_registers, display_list)]

Helper functions for implementing the display list entries

In [3]:
def getByteForAddress(address,scr):
    if 0x7000 <= address < 0x8000:
        return scr[address - 0x7000]
    return 0

def drawBlankLines(pixels, byte, y, num, scr):
    #pixels[x, y] = ImageColor.getrgb('#000000')
    return num

def drawScanLine(pixels, byte, y, num, scr):
    for x in range(0,40):
        byte = getByteForAddress(address+x, scr)
        left_nibble = byte >> 4 & 0x0F 
        right_nibble = byte & 0x0F
        for n in range(0,4):
            pixels[y][(x*8)+n] = col_register[left_nibble]
        for n in range(0,4):
            pixels[y][(x*8)+n+4] = col_register[right_nibble]
    return 1

dlist_dict = {
    0x70: (drawBlankLines,1,1),
    0x90: (drawBlankLines,1,1),
    0x46: (drawBlankLines,3,1),
    0x4F: (drawScanLine,  3,1),
}
def display_list_entry(dl):
    # Consume entries in the display list one by one.
    i = 0
    while i < len(dl):
        b = dl[i]
        if b not in dlist_dict:
            yield (None,None,None,None,None)
        draw, advance, lines = dlist_dict[b]
        address = (dl[i+2]<<8 | dl[i+1]) if advance > 1 else 0
        i += advance
        yield (b, draw, address, lines,i)


In [4]:
ROWS = 192
COLS = 320

pixel_history = []
#selected = [ram_history[x] for x in [188,236]]
for j, (scr, col_register, display_list) in enumerate(ram_history):
    pixels = [[0 for i in range(0,COLS)] for i in range(0,ROWS)]
    dlist = display_list_entry(display_list)
    x,y = 0,0
    while True:
        token, draw, address, num, i = next(dlist)
        if draw == None:
            break
        #print(hex(token),hex(address),num,i,display_list[i:i+2],len(display_list))
        y += draw(pixels, address, y, num, scr)
    pixel_history+= [pixels]

In [6]:
num_pixel_list = []
for i,h in enumerate(pixel_history):
    num_pixels = sum(flatten(h))
    num_pixel_list += [num_pixels]
num_pixel_list

[7168,
 7168,
 5184,
 9280,
 5184,
 2368,
 10960,
 2368,
 89088,
 22784,
 95232,
 101952,
 111840,
 101952,
 112000,
 125064,
 112000,
 0,
 120960,
 2660,
 2680,
 9552,
 9552,
 13280,
 13280,
 14784,
 14784,
 14112,
 14112,
 11264,
 11264,
 6672,
 6672,
 688,
 688,
 2784,
 2784,
 4480,
 4576,
 4480,
 6048,
 6208,
 6048,
 7488,
 7784,
 7488,
 8800,
 9192,
 8800,
 10192,
 10368,
 10192,
 0,
 10192,
 560,
 568,
 14352,
 25272,
 25608,
 33048,
 33128,
 33424,
 38288,
 38048,
 38128,
 38384,
 47000,
 46904,
 46844,
 46824,
 55944,
 55944,
 55848,
 54528,
 45408,
 54388,
 54388,
 53600,
 43776,
 52576,
 51368,
 42688,
 50624,
 0,
 0,
 890320,
 910800,
 911920,
 910800,
 0,
 7560,
 0,
 0,
 0,
 0,
 892880,
 901712,
 892880,
 0,
 8280,
 0,
 0,
 0,
 870840,
 870960,
 7424,
 7424,
 12600,
 12800,
 12544,
 19096,
 19456,
 19008,
 21632,
 22336,
 21632,
 25200,
 26184,
 25376,
 27240,
 27744,
 27048,
 0,
 27048,
 0,
 640,
 15616,
 26552,
 27136,
 41528,
 41984,
 48664,
 48960,
 58416,
 57864,
 6307

In [25]:
flatten = lambda l: [e for sublist in l for e in sublist]

pattern_breaks = []
for i,h in enumerate(pixel_history):
    num_pixels = sum(flatten(h))
    if num_pixels > 1:
        continue
    pattern_breaks += [i]

patterns = []
pp = 0
for p in pattern_breaks:
    patterns += [pixel_history[pp:p]]
    pp = p
len(patterns)

17

In [81]:
# Fix up the patterns
del patterns[2]
del patterns[3]
del patterns[4]
del patterns[5]
del patterns[7]
del patterns[9]
del patterns[11]
del patterns[13]


In [82]:
last = patterns[-1]
del patterns[-1]
patterns += [last[:16]]
patterns += [last[17:]]


In [26]:
def firstNonZero(l):
    for i,e in enumerate(l):
        if e > 0:
            return i
    return i

def getLeftMostPixelForPattern(pattern):
    return min([firstNonZero(r) for scr in pattern for r in scr])

def getRightMostPixelForPattern(pattern):
    return len(pattern[0][0]) - min([firstNonZero(list(reversed(r)))
                     for scr in pattern for r in scr])

def rotatePatternToCols(pattern):
    pattern_cols = []
    for scr in pattern:
        col_arr = [[0 for i in range(len(scr))] for j in range(len(scr[0]))]
        for i,r in enumerate(scr):
            for j,v in enumerate(r):
                col_arr[j][i] = v
        pattern_cols += [col_arr]
    return pattern_cols

def cullPattern(pattern):
    first_left = getLeftMostPixelForPattern(pattern)
    last_right = getRightMostPixelForPattern(pattern)
    width = last_right - first_left

    pattern_as_cols = rotatePatternToCols(pattern)
    first_top = getLeftMostPixelForPattern(pattern_as_cols)
    last_bottom = getRightMostPixelForPattern(pattern_as_cols)
    height = last_bottom - first_top
    
    culled = [[r[first_left:last_right] 
              for r in scr[first_top:last_bottom]]
              for scr in pattern]
    return culled

#cullPattern(patterns[0])
#patterns[0]

In [27]:
import matplotlib.pyplot as plt
from PIL import Image, ImageColor, ImageChops
from matplotlib.colors import LightSource
import colorspace_colors as cc
import numpy as np

plt.rcParams["figure.figsize"] = (80,80)
plt.rcParams['figure.dpi'] = 80

def explode(data):
    size = np.array(data.shape)*2
    data_e = np.zeros(size - 1, dtype=data.dtype)
    data_e[::2, ::2, ::2] = data
    return data_e

def createVoxelData(screens):
    # prepare some coordinates
    width = len(screens[0][0])
    height = len (screens[0])
    x, y, z = np.indices((width,height,len(screens)))
    voxelarray = np.zeros((width,height,len(screens)), dtype=bool)
    colorarray = np.empty(voxelarray.shape, dtype=object)
    for zp,screen in enumerate(screens):
        for yp, row in enumerate(list(reversed(screen))):
            for xp, col in enumerate(row):
                # Skip black, white and gray cells
                if col in [0,1,0x0c]:
                    continue
                cube = (x == xp) & (y == yp) & (z == zp)
                voxelarray |= cube
                color =  cc.color_value_to_html[col]
                colorarray[cube] = color+"c0"

    # upscale the above voxel image, leaving gaps
    filled_2 = explode(voxelarray)
    ecolors_2 = explode(colorarray)

    # Shrink the gaps
    x, y, z = np.indices(np.array(filled_2.shape) + 1).astype(float) // 2
    x[0::2, :, :] += 0.10
    y[:, 0::2, :] += 0.10
    z[:, :, 0::2] += 0.10
    x[1::2, :, :] += 0.90
    y[:, 1::2, :] += 0.90
    z[:, :, 1::2] += 0.90

    return ((x,y,z), filled_2, ecolors_2)

def fig2img(fig):
    """Convert a Matplotlib figure to a PIL Image and return it"""
    import io
    buf = io.BytesIO()
    fig.savefig(buf)
    buf.seek(0)
    img = Image.open(buf)
    return img

In [ ]:
# Select our pattern
import os
import gc

def trim(im):
    bg = Image.new(im.mode, im.size, im.getpixel((0,0)))
    diff = ImageChops.difference(im, bg)
    bbox = diff.getbbox()
    print(bbox)
    return im.crop(bbox)

for i, screens in enumerate([p for p in patterns if len(p) > 1]):
    screens = cullPattern(screens)
    for azim in [-45]:
        if os.path.isfile(f'colorspace_presets/preset{i+1}{azim}.png'):
            continue
        ((x,y,z), voxels, vcolors) = createVoxelData(screens)
        # and plot everything
        ax = plt.figure().add_subplot(projection='3d')
        ls = LightSource(45)
        ax.voxels(x,y,z,voxels, facecolors=vcolors, edgecolors=vcolors,lightsource=ls)
        ax.axis('off')
        ax.view_init(azim=azim, vertical_axis='y')
        print("fig2img")
        img = fig2img(plt.gcf())
        img = img.resize((int(img.width / 2), int(img.height / 2)), Image.NEAREST)
        
        newImage = []
        for item in img.getdata():
            if item[:3] == (255, 255, 255):
                newImage.append((255, 255, 255, 0))
            else:
                newImage.append(item)

        img.putdata(newImage)
        img = trim(img)
        img.save(f'colorspace_presets/preset{i+1}{azim}.png')
        plt.clf()
        del voxels, vcolors
        plt.close()
        gc.collect()


fig2img
(948, 1033, 2697, 2519)
fig2img
(683, 812, 2639, 2489)
